<a href="https://colab.research.google.com/github/nyp-sit/it3103/blob/main/week14/intent_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intent Recognition

In this practical, we will learn how to apply the HuggingFace Transformers library to our own Intent Recognition task for our chatbot.

####**NOTE: Be sure to set your runtime to a GPU instance!**

## Install the Hugging Face Transformers Library

Run the following cell below to install the transformers library.

In [ ]:
!pip install transformers==4.7

## Getting the data and prepare the data

In [ ]:
import pandas as pd

data_url = 'https://nyp-aicourse.s3.ap-southeast-1.amazonaws.com/datasets/airchat_intents.csv'
df = pd.read_csv(data_url)

df.head()

We noticed that there are two columns 'Label' and 'Text'. Let's just examine what are the different labels we have and how many samples we have for each labels.

In [ ]:
df['Label'].value_counts()

We can see that some labels have very few sample such as 'atis_meal', 'atis_airline#atis_flight_no', 'atis_cheapest', and so on. We so few samples, our model will have difficulty in learning any meaningful pattern from it. We will group these labels (with few samples) into a new label called 'others'.  

---



### Re-define our Classification Labels

Here we define the labels we are interested in classifying based on the original labels, and also we added a new label called 'Others'.
 

In [ ]:
# Create a list of unique labels that we will recognize.
#
sentence_labels = [
              "others",
              "atis_abbreviation",
              "atis_aircraft",
              "atis_airfare",
              "atis_airline",
              "atis_flight",
              "atis_flight_time",
              "atis_greeting",
              "atis_ground_service",
              "atis_quantity",
              "atis_yes",
              "atis_no"]

# This creates a reverse mapping dictionary of "label" -> index.
# 
sentence_labels_id_by_label = dict((t, i) for i, t in enumerate(sentence_labels))

Now we will map the previous labels to the few ones we specified in the cell above. We will also convert the text labels into numeric labels (e.g. others->0, atis_abbreviation->1, etc). We can use the `map()` function in dataframe to help us do that. We define a lambda function that do the mapping.

In [ ]:
df['Label'] = df['Label'].map(lambda label: 
                              sentence_labels_id_by_label[label] 
                              if label in sentence_labels_id_by_label 
                              else 0)

In [ ]:
# examine a few random samples 
df.sample(10)

### Split Our Data

We will now separate the texts and labels and call them all_texts and all_labels and we will split the dataset into training and validation set. We do a stratified split to ensure we have equal representation of different labels in both train and validation set.

In [ ]:
all_texts = df['Text']
all_labels = df['Label']

In [ ]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(all_texts, 
                                                                    all_labels, 
                                                                    test_size=0.2, 
                                                                    stratify=all_labels)

In [ ]:
train_labels.value_counts()/len(train_labels)

In [ ]:
val_labels.value_counts()/len(val_labels)

### Tokenize the text 

Before we can use the text for classification, we need to tokenize them. We will use Tokenizer of the pretrained model 'distilbert-base-uncased' as we will be fine-tunining on a pretrained model 'distilbert-base-uncased'. 


In [ ]:
## before we can feed the texts to tokenizer, we need to convert our texts into list of text string instead of 
## panda Series. We can do this by using to_list(). 

train_texts = train_texts.to_list()
train_labels = train_labels.to_list()
val_texts = val_texts.to_list()
val_labels = val_labels.to_list()

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(train_texts, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, padding=True, truncation=True)

Once we have the encodings, we will go ahead and create a tensorflow dataset, ready to be used to train our model. Since the HuggingFace pretrained model (the tensorflow version) is a Keras model, it can consume the tf.data dataset. 

In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

## Train Your Sentence Classification Model

Run the following cell to download the "distilbert-base-uncased" and perform fine-tuning training using the dataset that we have above.

In [ ]:
import numpy as np 

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {"acc": (preds == p.label_ids).mean()}

In [ ]:
from transformers import TFAutoModelForSequenceClassification, TFTrainer, TFTrainingArguments
from transformers.utils import logging as hf_logging

# We enable logging level to info and use default log handler and log formatting
hf_logging.set_verbosity_info()
hf_logging.enable_default_handler()
hf_logging.enable_explicit_format()

training_args = TFTrainingArguments(
    output_dir='./checkpoints',          # output directory
    num_train_epochs=2,                  # total number of training epochs
    logging_dir='./tblogs',              # directory for storing logs
    logging_steps=10,
    evaluation_strategy='epoch'         # run evaluation on validation after every epoch
)

# we need to use the strategy.scope() here to avoid error saying the weights were trained using different scope
# the numb_labels tells the classification model how many outputs we need. It will be as many unique labels we have
with training_args.strategy.scope():
    intent_classification_model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", 
                                                                                     num_labels=len(sentence_labels))

trainer = TFTrainer(
    model=intent_classification_model,   # the instantiated 🤗 Transformers model to be trained
    compute_metrics=compute_metrics,
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)



In [ ]:
trainer.train()

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir tblogs

### Saving the Model

When you training has completed, run the following cell to save your model.

Remember to download the model from Google Colab if you want to use later.

In [ ]:
# Save the model
#
#intent_classification_model.save("intentclassification_model")
intent_classification_model.save_pretrained("intent_model")

### Evaluating the Model

Run the following code to perform interference for the entire training and validation data set.


In [ ]:
preds = trainer.predict(val_dataset)

In [ ]:
tf_predictions = tf.nn.softmax(preds.predictions, axis=-1)

In [ ]:
y_preds = np.argmax(tf_predictions, axis=-1)


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(preds.label_ids, y_preds))

## Putting Our Model to the Test

Run the following cell to create the necessary classes and functions to load our model and perform inference.


In [ ]:
# Import the necessary libraries
#
from transformers import (
    AutoTokenizer,
    TFAutoModelForSequenceClassification
)
import numpy as np

# Create the DistilBERT tokenizer
#
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Define a function to perform inference on a single input text.
# 
def infer_intent(model, text):
    # Passes the text into the tokenizer
    #
    input = tokenizer(text, truncation=True, padding=True, return_tensors="tf")
    print(input)

    # Sends the result from the tokenizer into our classification model
    #
    output = model(input)

    # Extract the output logits and convert to softmax 
    # Find the classification index with the highest value.
    #  
    pred_label = np.argmax(tf.nn.softmax(output.logits, axis=-1))

    return pred_label

# Create a list of unique labels that we will recognize.
# Obviously this has to match what we trained our model with
# earlier.
#
sentence_labels = [
              "others",
              "atis_abbreviation",
              "atis_aircraft",
              "atis_airfare",
              "atis_airline",
              "atis_flight",
              "atis_flight_time",
              "atis_greeting",
              "atis_ground_service",
              "atis_quantity",
              "atis_yes",
              "atis_no"]

# Load the saved model file
#
intent_model = TFAutoModelForSequenceClassification.from_pretrained("intent_model")

text = input()

print (sentence_labels[infer_intent(intent_model, text)])